In [43]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from tqdm import tqdm

In [44]:
class Distribution(nn.Module):
    def __init__(self, size=1000):
        super().__init__()
        self.mu = nn.Parameter(torch.tensor(0.0), requires_grad=True)
        self.std = nn.Parameter(torch.tensor(1.0), requires_grad=True)
        self.size = size
    def forward(self):
        return self.mu.detach().numpy() + self.std.detach().numpy() * np.random.normal(0, 1, size=self.size)

class LinearRegression(nn.Module):
    def __init__(self, size=1000):
        super().__init__()
        self.m = Distribution(size=size)
        self.b = Distribution(size=size)
    def forward(self, x, y):
        m = self.m()
        print(type(m))
        b = self.b()
        
        def loss(x_i, y_i, m_j, b_j):
            return (y_i - (m_j * x_i - b_j)) ** 2

        def loss_val(x_i, y_i):
            vals = np.vectorize(loss)(x_i, y_i, m, b)
            return (1 / len(m)) * np.sum(vals)
    
        vals = np.vectorize(loss_val)(x, y)
        total_loss = np.sum(vals)
        
        # Minimize loss by maximizing this value
        return 1 / total_loss

In [45]:
def training(x, y, epochs=5):
    model = LinearRegression()
    optimizer = optim.Adam(model.parameters())
    
    for epoch in tqdm(range(epochs), desc="Training..."):
        for i in range(len(x)):
            # Zeros gradiant for training
            optimizer.zero_grad()
            
            # Calculates likelihood
            loglik = model(x[i], y[i])
            e = -torch.mean(loglik)
            
            # Updates parameters
            e.backward()
            optimizer.step()
        
    return model

In [46]:
df = pd.read_csv('data/Advertising.csv').drop('Unnamed: 0', axis=1)
df

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9
...,...,...,...,...
195,38.2,3.7,13.8,7.6
196,94.2,4.9,8.1,9.7
197,177.0,9.3,6.4,12.8
198,283.6,42.0,66.2,25.5


In [47]:
x = df.TV.to_numpy()
y = df.Sales.to_numpy()

In [48]:
res = training(x, y, epochs=10)
res

Training...:   0%|          | 0/10 [00:00<?, ?it/s]

<class 'numpy.ndarray'>


TypeError: mean(): argument 'input' (position 1) must be Tensor, not numpy.float64